In [18]:
from torch_geometric_temporal.dataset import EnglandCovidDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

In [19]:
loader = EnglandCovidDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [20]:
dataset[0]

Data(x=[129, 8], edge_index=[2, 2158], edge_attr=[2158], y=[129])

In [21]:
dataset[1]

Data(x=[129, 8], edge_index=[2, 1743], edge_attr=[1743], y=[129])

In [22]:
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = METRLADatasetLoader()

dataset = loader.get_dataset(num_timesteps_in = 12, num_timesteps_out = 12)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [23]:
dataset[0]

Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])

In [24]:
from torch_geometric_temporal.dataset import PemsBayDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = PemsBayDatasetLoader()

dataset = loader.get_dataset(num_timesteps_in = 12, num_timesteps_out = 12)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [25]:
train_dataset[0]

Data(x=[325, 2, 12], edge_index=[2, 2694], edge_attr=[2694], y=[325, 2, 12])

In [26]:
train_dataset[1]

Data(x=[325, 2, 12], edge_index=[2, 2694], edge_attr=[2694], y=[325, 2, 12])

In [27]:
import numpy as np

In [28]:
A = np.load('/home/zinanzheng/project/insitu/baseline/spatial_temporal/data/pems_adj_mat.npy')
X = np.load('/home/zinanzheng/project/insitu/baseline/spatial_temporal/data/pems_node_values.npy').transpose(
    (1, 2, 0)
)

In [35]:
X.shape

(325, 2, 52105)

In [30]:
A

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.9605087,
        0.6060658],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.7730252],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]], dtype=float32)

In [33]:
A = np.load('/home/zinanzheng/project/insitu/data/step2_generate_data/Europe_sparse_adjacency_matrix.npy')

In [34]:
A.shape

(255, 255)

In [1]:
from torch_geometric_temporal.dataset import Weather
from torch_geometric_temporal.signal import temporal_signal_split

loader = Weather(area="Europe")

dataset = loader.get_dataset(num_timesteps_in = 12, num_timesteps_out = 12)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [2]:
train_dataset[0]

Data(x=[255, 1, 12], edge_index=[2, 2805], edge_attr=[2805], y=[255, 1, 12])

In [1]:
import numpy as np
X = np.load('/home/zinanzheng/project/insitu/data/step2_generate_data/data/America_node_values.npy')

In [2]:
X.shape

(135, 1, 6575)

In [4]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [5]:
train_dataset[0]

Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [7]:
from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = WikiMathsDatasetLoader()

dataset = loader.get_dataset(lags=14)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.5)
train_dataset[0]

Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])

In [3]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/200 [00:00<?, ?it/s]


RuntimeError: Tensors must have same number of dimensions: got 3 and 2